In [ ]:
# Libraries

# Imports and global config
from __future__ import annotations

# Standard library
import os
import glob
import csv
import time
import warnings
import re
import itertools
from pathlib import Path

# Core data science
import numpy as np
import pandas as pd

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
from pymoo.visualization.scatter import Scatter

# Statistics and Factor Analysis
from scipy import stats
from scipy.stats import kruskal, mannwhitneyu, shapiro, levene
from scipy.linalg import cholesky, det, eigvals, inv
from factor_analyzer import FactorAnalyzer
from statsmodels.multivariate.manova import MANOVA

# Decomposition and Discriminant Analysis
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

# Ensembles and Tree-based Methods
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier

# Feature Selection and Importance
from boruta import BorutaPy
from sklearn.feature_selection import RFE, f_classif, mutual_info_classif
from sklearn.inspection import permutation_importance

# Linear/Logistic Regression and Classification
from sklearn.linear_model import LinearRegression, LogisticRegression

# Model Evaluation and Cross-Validation
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import (
    KFold, LeaveOneOut, StratifiedKFold,
    cross_val_predict, cross_validate, cross_val_score,
    GridSearchCV, learning_curve, train_test_split
)

# Probabilistic and Distance-based Methods
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

# Neural Networks and Pipelines
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import Pipeline, make_pipeline

# Preprocessing
from sklearn.preprocessing import LabelEncoder, StandardScaler

# Boosting Libraries
import xgboost as xgb
from xgboost import XGBClassifier, plot_importance
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier

# Genetic Algorithms and Bayesian Optimization
from deap import base, creator, tools, algorithms
import optuna
from optuna.exceptions import ExperimentalWarning

# Explainability/Model Interpretation
import lime
import lime.lime_tabular
import shap

# Design of Experiments (DOE)
from pyDOE2 import fracfact

# General: suppress non-critical warnings
optuna.logging.set_verbosity(optuna.logging.ERROR)
warnings.filterwarnings("ignore", category=ExperimentalWarning)
warnings.filterwarnings("ignore")

In [ ]:
# Dataset loading


# List of dataset base filenames (without extension)
dataset_names = [
    "statlog_australian_credit_approval",
    "wine_quality_red",
    "diabetic_retinopathy_debrecen",
    "breast_cancer_wisconsin",
    "burst_header_packet",
    "seismic_bumps",
    "iranian_churn",
    "qsar_biodegradation",
    "image_segmentation",
    "wine_quality_white",
    "steel_plates_faults",
    "gender_gap_spanish_wp",
    "waveform_database_generator"
]

# Dictionary to hold each loaded DataFrame keyed by dataset name
datasets = {}

for name in dataset_names:
    # Load each dataset from CSV into a DataFrame
    df = pd.read_csv(f"datasets/{name}.csv")

    # Drop any rows containing at least one missing value
    # (ensures consistent model input across datasets)
    df = df.dropna(axis=0, how="any")

    # Store the cleaned DataFrame
    datasets[name] = df

# Container for per-dataset outputs/results (e.g., metrics, models, artifacts)
outputs = {}

In [ ]:
# Data bases (Original, PCA, and PCFA)


# Iterate over all datasets
for i, (name, original_df) in enumerate(datasets.items(), start=1):
    # Remove rows containing any NaNs and reset index
    original_df = original_df.dropna().reset_index(drop=True)

    # Remove constant columns to avoid degenerate components (inf/NaN in decompositions)
    X = original_df.drop(columns="y")
    X = X.loc[:, X.nunique() > 1]
    y = original_df["y"].values

    # Standardize features (mean=0, std=1) for PCA/PCFA
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    # PCA to determine the number of components (retain >= 85% cumulative variance)
    pca = PCA(n_components=0.85, svd_solver="full")
    X_pca = pca.fit_transform(X_scaled)
    n_components_pca = X_pca.shape[1]

    # PCA scores DataFrame
    pca_df = pd.DataFrame(
        X_pca,
        columns=[f"PC{j+1}" for j in range(n_components_pca)],
        index=original_df.index
    )
    pca_df["y"] = y

    # PCFA (PCA-informed Factor Analysis) with Varimax rotation
    # Start with at least 2 factors, using the PCA-derived count as baseline
    n_components_fa = max(n_components_pca, 2)
    fa = FactorAnalyzer(
        n_factors=n_components_fa,
        rotation="varimax",
        method="principal"
    )
    try:
        X_factors = fa.fit_transform(X_scaled)
    except Exception:
        # Fallback: if FA fails (e.g., due to rank deficiency), reduce number of factors
        n_components_fa = min(n_components_fa, X_scaled.shape[1] - 1)
        fa = FactorAnalyzer(
            n_factors=n_components_fa,
            rotation="varimax",
            method="principal"
        )
        X_factors = fa.fit_transform(X_scaled)

    # PCFA scores DataFrame
    fa_df = pd.DataFrame(
        X_factors,
        columns=[f"F{j+1}" for j in range(n_components_fa)],
        index=original_df.index
    )
    fa_df["y"] = y

    # Store outputs for this dataset
    ds_key = f"dataset_{i}"
    outputs.setdefault(ds_key, {})
    outputs[ds_key]["original"]   = original_df
    outputs[ds_key]["pca_scores"] = pca_df
    outputs[ds_key]["fa_scores"]  = fa_df


# Persist transformed datasets to Excel files

# Keep sheet names within Excel's 31-character limit
enums = list(datasets.keys())

# Original
with pd.ExcelWriter("df_original.xlsx", engine="openpyxl") as writer:
    for i, name in enumerate(enums, start=1):
        df_orig = outputs[f"dataset_{i}"]["original"]
        sheet = f"{i}_{name}"[:31]
        df_orig.to_excel(writer, sheet_name=sheet, index=False)

# PCA
with pd.ExcelWriter("df_pca.xlsx", engine="openpyxl") as writer:
    for i, name in enumerate(enums, start=1):
        df_pca = outputs[f"dataset_{i}"]["pca_scores"]
        sheet = f"{i}_{name}"[:31]
        df_pca.to_excel(writer, sheet_name=sheet, index=False)

# PCFA (FA)
with pd.ExcelWriter("df_fa.xlsx", engine="openpyxl") as writer:
    for i, name in enumerate(enums, start=1):
        df_fa = outputs[f"dataset_{i}"]["fa_scores"]
        sheet = f"{i}_{name}"[:31]
        df_fa.to_excel(writer, sheet_name=sheet, index=False)

In [ ]:
# Dataset selection and factorial design

# Config
DATASET  = 'original'  # options: 'original', 'pca', 'fa'
seed      = 42
fraction  = 1.0        # 1.0 = full factorial before hitting max_iter
max_iter  = 256        # desired maximum number of iterations (rows) per design

def create_full_and_frac_designs(
    num_features: int,
    fraction: float = 1.0,
    seed: int | None = None,
    prefix: str | list[str] = 'F',
    max_iter: int | None = None
) -> tuple[pd.DataFrame, pd.DataFrame]:
    """
    Create full and fractional factorial designs with two levels (-1, +1).

    If the full design (2**num_features) would exceed max_iter, generate a
    random fractional sample directly with size = max_iter.

    Parameters
    ----------
    num_features : int
        Number of factors (features) to include in the design.
    fraction : float, default=1.0
        Fraction of the full design to keep in the fractional design (0 < fraction <= 1).
    seed : int | None
        Random seed for reproducibility.
    prefix : str | list[str], default='F'
        Column names. If str, columns will be prefix + index (e.g., F1, F2, ...).
        If list, it is used as-is.
    max_iter : int | None
        Cap the number of design rows. If None, no cap is applied.

    Returns
    -------
    full_df : pd.DataFrame
        Full factorial design (possibly capped by max_iter).
    frac_df : pd.DataFrame
        Fractional design (random subset) honoring `fraction` and `max_iter`.
    """
    rng = np.random.default_rng(seed)

    # Build column names
    if isinstance(prefix, list):
        cols = prefix
    else:
        cols = [f"{prefix}{i+1}" for i in range(num_features)]

    total_pts = 2 ** num_features

    # If a full design would exceed max_iter, create a random fractional sample directly
    if max_iter is not None and total_pts > max_iter:
        n_sel = max_iter
        mat = rng.choice([-1, 1], size=(n_sel, num_features))
        full_df = pd.DataFrame(mat, columns=cols)
        full_df.insert(0, "Iteration", np.arange(1, n_sel + 1))
        frac_df = full_df.copy()
    else:
        # Full factorial matrix
        mat = np.array(np.meshgrid(*([[-1, 1]] * num_features))).T.reshape(-1, num_features)
        full_df = pd.DataFrame(mat, columns=cols)
        full_df.insert(0, "Iteration", np.arange(1, len(full_df) + 1))

        # Optionally cap the full design to max_iter
        if max_iter is not None and len(full_df) > max_iter:
            sel = rng.choice(full_df.index, size=max_iter, replace=False)
            full_df = full_df.loc[sel].reset_index(drop=True)
            full_df["Iteration"] = np.arange(1, len(full_df) + 1)

        # Fractional design (random subset of the full design)
        if 0 < fraction < 1.0:
            n_sel = max(1, int(len(full_df) * fraction))
            if max_iter is not None:
                n_sel = min(n_sel, max_iter)
            idxs = rng.choice(full_df.index, size=n_sel, replace=False)
            frac_df = full_df.loc[idxs].reset_index(drop=True)
            frac_df["Iteration"] = np.arange(1, len(frac_df) + 1)
        else:
            frac_df = full_df.copy()

    return full_df, frac_df

# Iterate through each loaded dataset
for i, (name, _) in enumerate(datasets.items(), start=1):
    ds_key = f"dataset_{i}"
    orig   = outputs[ds_key]["original"]
    pca    = outputs[ds_key]["pca_scores"]
    fa_df  = outputs[ds_key]["fa_scores"]

    # Select the working DataFrame and set column prefixes accordingly
    if DATASET == 'original':
        df_used = orig
        X_full  = df_used.drop(columns='y')
        prefix  = list(X_full.columns)
    elif DATASET == 'pca':
        df_used = pca
        X_full  = df_used.drop(columns='y')
        prefix  = [f"PC{j+1}" for j in range(X_full.shape[1])]
    else:
        df_used = fa_df
        X_full  = df_used.drop(columns='y')
        prefix  = [f"F{j+1}" for j in range(X_full.shape[1])]

    y_full       = df_used['y']
    num_features = X_full.shape[1]

    # Create full and fractional designs honoring max_iter
    full_df, frac_df = create_full_and_frac_designs(
        num_features, fraction, seed,
        prefix=prefix, max_iter=max_iter
    )

    outputs[ds_key]["design"] = frac_df

# Persist designs to an Excel workbook (one sheet per dataset)
with pd.ExcelWriter("df_designs.xlsx", engine="openpyxl") as writer:
    for i, (name, _) in enumerate(datasets.items(), start=1):
        df_design = outputs[f"dataset_{i}"]["design"]
        sheet     = f"{i}_{name}_{DATASET}"[:31]  # Excel sheet name limit is 31 chars
        df_design.to_excel(writer, sheet_name=sheet, index=False)

In [ ]:
# ML models: training & evaluation


# Iterate over each dataset in `datasets`
for i, (name, _) in enumerate(datasets.items(), start=1):
    ds_key = f"dataset_{i}"
    
    # Select the input DataFrame according to DATASET flag
    df_used = {
        'original': outputs[ds_key]["original"],
        'pca'     : outputs[ds_key]["pca_scores"],
        'fa'      : outputs[ds_key]["fa_scores"]
    }[DATASET]
    
    # Prepare X, y and the factorial design
    X_full    = df_used.drop(columns="y")
    y_full    = df_used["y"]
    y         = LabelEncoder().fit_transform(y_full)
    X         = X_full.copy()
    design_df = outputs[ds_key]["design"]
    
    # Define models and scoring metrics
    MODELS = {
        'KNN':  KNeighborsClassifier(n_jobs=-1),
        'LR':   LogisticRegression(max_iter=1000, n_jobs=-1),
        'LDA':  LinearDiscriminantAnalysis(),
        'NB':   GaussianNB(),
        'SVM':  SVC(random_state=42),
        'RF':   RandomForestClassifier(random_state=42, n_jobs=-1),
        'GB':   GradientBoostingClassifier(random_state=42),
        'XGB':  XGBClassifier(use_label_encoder=False, random_state=42, n_jobs=-1),
        'LGBM': LGBMClassifier(random_state=42, n_jobs=-1, verbose=-1),
        'MLP':  MLPClassifier(random_state=42, max_iter=500, early_stopping=True, n_iter_no_change=10)
    }
    SCORING = {
        'A' : 'accuracy',
        'P' : 'precision_weighted',
        'R' : 'recall_weighted',
        'F1': 'f1_weighted'
    }

    all_results = []
    all_times   = []

    # Evaluate each design iteration (row) produced by DOE
    for idx, design_row in design_df.iterrows():
        # Select columns flagged with +1 in the design (and that exist in X)
        sel_cols    = [c for c, v in design_row.items() if v == 1 and c in X.columns]
        perf_scores = {}
        time_scores = {}

        if not sel_cols:
            # No features selected: fill with NaNs for metrics and time
            for m in MODELS:
                for abbr in SCORING:
                    perf_scores[f"{m}({abbr})"] = np.nan
                time_scores[f"{m}_time"] = np.nan
        else:
            X_sub = X[sel_cols]
            cv    = StratifiedKFold(n_splits=5, shuffle=True, random_state=idx)

            # Cross-validate each model on the selected subset of features
            for mname, model in MODELS.items():
                start  = time.time()
                cv_res = cross_validate(
                    model,
                    X_sub, y,
                    cv=cv,
                    scoring=SCORING,
                    n_jobs=-1,
                    return_train_score=False
                )
                elapsed = time.time() - start

                # Store mean test scores for each metric
                for abbr in SCORING:
                    perf_scores[f"{mname}({abbr})"] = cv_res[f"test_{abbr}"].mean()
                time_scores[f"{mname}_time"] = elapsed

        all_results.append(perf_scores)
        all_times.append(time_scores)

    # Build results/time DataFrames for this dataset
    results_df = pd.DataFrame(all_results)
    times_df   = pd.DataFrame(all_times)

    # Enforce a consistent column order
    metric_cols = [f"{m}({abbr})" for m in MODELS for abbr in SCORING]
    time_cols   = [f"{m}_time"   for m in MODELS]
    results_df  = results_df[metric_cols]
    times_df    = times_df[time_cols]

    # Store in outputs
    outputs[ds_key]["results"] = results_df
    outputs[ds_key]["times"]   = times_df

# Persist per-design results to Excel workbooks
with pd.ExcelWriter("df_ml_results.xlsx", engine="openpyxl") as writer:
    for i, (name, _) in enumerate(datasets.items(), start=1):
        df_res = outputs[f"dataset_{i}"]["results"]
        sheet  = f"{i}_{name}_{DATASET}"[:31]  # Excel sheet name limit is 31 characters
        df_res.to_excel(writer, sheet_name=sheet, index=False)

with pd.ExcelWriter("df_ml_times.xlsx", engine="openpyxl") as writer:
    for i, (name, _) in enumerate(datasets.items(), start=1):
        df_time = outputs[f"dataset_{i}"]["times"]
        sheet   = f"{i}_{name}_{DATASET}"[:31]
        df_time.to_excel(writer, sheet_name=sheet, index=False)

In [ ]:
# Performance metrics computation


# Iterate over each dataset to compute statistics
for i, (name, _) in enumerate(datasets.items(), start=1):
    ds_key      = f"dataset_{i}"
    # Ensure numeric conversion; invalid values become NaN
    results_df  = outputs[ds_key]["results"].copy().apply(pd.to_numeric, errors='coerce')
    
    # Detailed descriptive statistics for each column (Model(Metric))
    stat_names = ['Mean', 'Median', 'Range', 'Max', 'Min', 'Std Dev']
    descriptive_stats_df = pd.DataFrame({
        'Mean'   : results_df.mean(),
        'Median' : results_df.median(),
        'Range'  : results_df.max() - results_df.min(),
        'Max'    : results_df.max(),
        'Min'    : results_df.min(),
        'Std Dev': results_df.std()
    })[stat_names]
    
    # Aggregated statistics per model (averaging A, P, R, F1 across iterations)
    metrics_to_average = ['A','P','R','F1']
    model_prefixes = sorted({col.split('(')[0] for col in results_df.columns})
    agg_stats = {}
    for prefix in model_prefixes:
        # Collect only existing metric columns for the current model
        cols = [f"{prefix}({m})" for m in metrics_to_average if f"{prefix}({m})" in results_df]
        if cols:
            avg_per_iter = results_df[cols].mean(axis=1)  # Average across metrics for each iteration
            agg_stats[prefix] = [
                avg_per_iter.mean(),                      # Mean performance across all iterations
                avg_per_iter.median(),                    # Median performance
                avg_per_iter.max() - avg_per_iter.min(),  # Range
                avg_per_iter.max(),                       # Max performance
                avg_per_iter.min(),                       # Min performance
                avg_per_iter.std()                        # Standard deviation
            ]
        else:
            agg_stats[prefix] = [np.nan]*len(stat_names)
    
    # Create a DataFrame for aggregated statistics sorted by Mean in descending order
    final_summary_df = (pd.DataFrame.from_dict(
        agg_stats, orient='index', columns=stat_names
    ).sort_values('Mean', ascending=False).T)
    
    # Store the computed statistics
    outputs[ds_key]["descriptive_stats"] = descriptive_stats_df
    outputs[ds_key]["summary_stats"]     = final_summary_df


# Save statistics to Excel files
with pd.ExcelWriter("df_descriptive_stats.xlsx", engine="openpyxl") as writer:
    for i, (name, _) in enumerate(datasets.items(), start=1):
        df_detail = outputs[f"dataset_{i}"]["descriptive_stats"]
        sheet     = f"{i}_{name}_{DATASET}"[:31]  # Limit sheet name to Excel's 31-character max
        df_detail.to_excel(writer, sheet_name=sheet, index=True)

with pd.ExcelWriter("df_summary_stats.xlsx", engine="openpyxl") as writer:
    for i, (name, _) in enumerate(datasets.items(), start=1):
        df_summary = outputs[f"dataset_{i}"]["summary_stats"]
        sheet      = f"{i}_{name}_{DATASET}"[:31]
        df_summary.to_excel(writer, sheet_name=sheet, index=True)

In [ ]:
# Best results selection


# Iterate through each dataset to determine the best iterations according to different criteria
for i, (name, _) in enumerate(datasets.items(), start=1):
    ds_key     = f"dataset_{i}"
    results_df = outputs[ds_key]["results"].copy()
    design_df  = outputs[ds_key]["design"].copy()
    
    # Identify performance columns (those with the format Model(Metric))
    perf_cols = [c for c in results_df.columns if '(' in c and ')' in c]
    perf_data = results_df[perf_cols].apply(pd.to_numeric, errors='coerce')
    
    # Compute per-iteration statistics
    stats = pd.DataFrame({
        'Mean'   : perf_data.mean(axis=1),
        'Median' : perf_data.median(axis=1),
        'Range'  : perf_data.max(axis=1) - perf_data.min(axis=1),
        'Max'    : perf_data.max(axis=1),
        'Min'    : perf_data.min(axis=1),
        'Std Dev': perf_data.std(axis=1)
    }, index=perf_data.index)
    
    # Identify the iteration index for each "best" criterion
    best_criteria = {
        'Max of Mean'    : stats['Mean'].idxmax(),      # Iteration with highest mean score
        'Max of Median'  : stats['Median'].idxmax(),    # Iteration with highest median score
        'Min of Range'   : stats['Range'].idxmin(),     # Iteration with most consistent results (smallest range)
        'Max of Max'     : stats['Max'].idxmax(),       # Iteration with highest single metric
        'Max of Min'     : stats['Min'].idxmax(),       # Iteration with best worst-case metric
        'Min of Std Dev' : stats['Std Dev'].idxmin()    # Iteration with lowest variability
    }
    
    # Average performance per algorithm for each iteration
    algorithm_means_df = perf_data.groupby(lambda c: c.split('(')[0], axis=1).mean()
    
    # Prepare iteration number, number of features, and feature list string
    iter_numbers    = design_df['Iteration']
    feature_cols    = [c for c in design_df.columns if c != 'Iteration']
    feature_strings = (
        design_df[feature_cols]
        .astype(int)
        .apply(lambda row: ','.join(row.index[row == 1]), axis=1)
    )
    
    # Build final DataFrame with the best results for each criterion
    rows = []
    for criterion, idx in best_criteria.items():
        num_feats = int((design_df.loc[idx, feature_cols] == 1).sum())
        row = {
            'Criterion'      : criterion,
            'Original Index' : idx,
            'Iteration'      : iter_numbers.at[idx],
            'Num Features'   : num_feats,
            'Features'       : feature_strings.at[idx]
        }
        row.update(algorithm_means_df.loc[idx].to_dict())
        rows.append(row)
    
    best_results_df = pd.DataFrame(rows)
    
    # Store results in outputs
    outputs[ds_key]["best_results"] = best_results_df


# Save best results to Excel file
with pd.ExcelWriter("df_best_results.xlsx", engine="openpyxl") as writer:
    for i, (name, _) in enumerate(datasets.items(), start=1):
        sheet = f"{i}_{name}_{DATASET}"[:31]  # Excel sheet name limit
        outputs[f"dataset_{i}"]["best_results"].to_excel(
            writer, sheet_name=sheet, index=False
        )

In [ ]:
# Best single iteration selection

# Iterate over each dataset
for i, (name, _) in enumerate(datasets.items(), start=1):
    ds_key     = f"dataset_{i}"
    results_df = outputs[ds_key]["results"].copy()
    design_df  = outputs[ds_key]["design"].copy()
    
    # Identify performance columns (metrics) and feature selection columns
    perf_cols    = [c for c in results_df.columns if '(' in c and ')' in c]
    feature_cols = [c for c in design_df.columns if c != 'Iteration']

    # Calculate iteration-level statistics based on performance metrics
    perf_data = results_df[perf_cols].apply(pd.to_numeric, errors='coerce')
    stats = pd.DataFrame({
        'Mean'   : perf_data.mean(axis=1),                               # Mean performance across all metrics
        'Median' : perf_data.median(axis=1),                             # Median performance across all metrics
        'Range'  : perf_data.max(axis=1) - perf_data.min(axis=1),        # Spread between best and worst metric
        'Max'    : perf_data.max(axis=1),                                # Best single metric value
        'Min'    : perf_data.min(axis=1),                                # Worst single metric value
        'Std Dev': perf_data.std(axis=1)                                 # Variability of metrics
    }, index=perf_data.index)

    # Determine candidate iterations for "best" according to multiple criteria
    best_criteria = {
        'Max of Mean'   : stats['Mean'].idxmax(),       # Iteration with highest mean
        'Max of Median' : stats['Median'].idxmax(),     # Iteration with highest median
        'Min of Range'  : stats['Range'].idxmin(),      # Iteration with smallest range
        'Max of Max'    : stats['Max'].idxmax(),        # Iteration with highest single metric
        'Max of Min'    : stats['Min'].idxmax(),        # Iteration with highest worst-case
        'Min of Std Dev': stats['Std Dev'].idxmin()     # Iteration with lowest variability
    }

    # Select and break ties: choose iteration that appears most frequently among criteria
    sel_idxs   = [v for v in best_criteria.values() if pd.notna(v)]
    counts     = pd.Series(sel_idxs).value_counts()
    candidates = counts[counts == counts.max()].index.tolist()
    if len(candidates) > 1:
        avg_perf   = perf_data.loc[candidates].mean(axis=1)  # Tie-breaker: choose with highest average performance
        chosen_idx = avg_perf.idxmax()
    else:
        chosen_idx = candidates[0]

    # Extract details for the chosen iteration
    iter_num  = design_df.at[chosen_idx, 'Iteration']
    row_feat  = design_df.loc[chosen_idx, feature_cols].astype(int)
    features  = ','.join(row_feat.index[row_feat == 1])
    num_feats = int((row_feat == 1).sum())
    metrics   = perf_data.loc[chosen_idx].astype(float)

    # Build final DataFrame containing the best iteration's details and metrics
    best_results_df = pd.DataFrame([{
        'Original Index': chosen_idx,
        'Iteration'     : iter_num,
        'Num Features'  : num_feats,
        'Features'      : features,
        **metrics.to_dict()
    }])

    # Store in outputs
    outputs[ds_key]["best_iteration"] = best_results_df


# Save each best iteration to an Excel file
with pd.ExcelWriter("df_best_iterations.xlsx", engine="openpyxl") as writer:
    for i, (name, _) in enumerate(datasets.items(), start=1):
        sheet = f"{i}_{name}_{DATASET}"[:31]  # Excel sheet name limit
        outputs[f"dataset_{i}"]["best_iteration"] \
            .to_excel(writer, sheet_name=sheet, index=False)

In [ ]:
# Hyperparameter optimization with Optuna

# CV and scoring configuration
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
SCORING = {
    'A' : 'accuracy',
    'P' : 'precision_weighted',
    'R' : 'recall_weighted',
    'F1': 'f1_weighted'
}

# Factory that builds an Optuna objective for a given model
def make_objective(X_top, y_top, model_name):
    def objective(trial):
        # Hyperparameter suggestions per model
        if model_name == 'KNN':
            params = {
                'n_neighbors': trial.suggest_int('n_neighbors', 1, 30),
                'weights': trial.suggest_categorical('weights', ['uniform','distance'])
            }
            clf = KNeighborsClassifier(**params, n_jobs=-1)

        elif model_name == 'LR':
            params = {
                'C': trial.suggest_loguniform('C', 1e-3, 1e3),
                'penalty': 'l2',
                'solver': 'lbfgs'
            }
            clf = LogisticRegression(**params, max_iter=1000, n_jobs=-1)

        elif model_name == 'LDA':
            clf = LinearDiscriminantAnalysis(solver='svd')

        elif model_name == 'NB':
            params = {
                'var_smoothing': trial.suggest_loguniform('var_smoothing', 1e-12, 1e-6)
            }
            clf = GaussianNB(**params)

        elif model_name == 'SVM':
            params = {
                'C': trial.suggest_loguniform('C', 1e-3, 1e3),
                'gamma': trial.suggest_categorical('gamma', ['scale','auto'])
            }
            clf = SVC(**params, random_state=42)

        elif model_name == 'RF':
            params = {
                'n_estimators': trial.suggest_int('n_estimators', 100, 500),
                'max_depth': trial.suggest_int('max_depth', 2, 32)
            }
            clf = RandomForestClassifier(**params, random_state=42, n_jobs=-1)

        elif model_name == 'GB':
            params = {
                'n_estimators': trial.suggest_int('n_estimators', 50, 300),
                'learning_rate': trial.suggest_loguniform('learning_rate', 1e-3, 1.0),
                'max_depth': trial.suggest_int('max_depth', 2, 10)
            }
            clf = GradientBoostingClassifier(**params, random_state=42)

        elif model_name == 'XGB':
            params = {
                'n_estimators': trial.suggest_int('n_estimators', 50, 500),
                'learning_rate': trial.suggest_loguniform('learning_rate', 1e-3, 1.0),
                'max_depth': trial.suggest_int('max_depth', 2, 16)
            }
            clf = XGBClassifier(**params, use_label_encoder=False,
                                random_state=42, n_jobs=-1)

        elif model_name == 'LGBM':
            params = {
                'n_estimators': trial.suggest_int('n_estimators', 50, 500),
                'num_leaves': trial.suggest_int('num_leaves', 8, 128),
                'learning_rate': trial.suggest_loguniform('learning_rate', 1e-3, 1.0)
            }
            clf = LGBMClassifier(**params, random_state=42, n_jobs=-1, verbose=-1)

        elif model_name == 'MLP':
            params = {
                'hidden_layer_sizes': trial.suggest_categorical(
                    'hidden_layer_sizes', [(50,), (100,), (50, 50)]
                ),
                'alpha': trial.suggest_loguniform('alpha', 1e-5, 1e-1),
                'learning_rate_init': trial.suggest_loguniform('learning_rate_init', 1e-4, 1e-1)
            }
            clf = MLPClassifier(**params, random_state=42,
                                max_iter=500, early_stopping=True, n_iter_no_change=10)
        else:
            raise ValueError(f"Unsupported model: {model_name}")

        # Optimize weighted F1 via cross-validation
        scores = cross_validate(
            clf, X_top, y_top, cv=cv,
            scoring='f1_weighted', n_jobs=-1,
            return_train_score=False
        )
        return scores['test_score'].mean()
    return objective

# Tuning loop per dataset
for i, (name, _) in enumerate(datasets.items(), start=1):
    ds_key = f"dataset_{i}"
    
    # Select the proper DataFrame according to previous choice
    df_used = {
        'original': outputs[ds_key]['original'],
        'pca'     : outputs[ds_key]['pca_scores'],
        'fa'      : outputs[ds_key]['fa_scores']
    }[DATASET]
    
    design_df = outputs[ds_key]['design']
    best_idx  = outputs[ds_key]['best_iteration'].at[0, 'Original Index']
    
    # Build X_top, y_top based on the best iteration's selected features
    feat_cols = [c for c in design_df.columns if c != 'Iteration']
    selected  = [c for c in feat_cols if design_df.at[best_idx, c] == 1]
    X_top     = df_used.drop(columns='y')[selected]
    y_top     = LabelEncoder().fit_transform(df_used['y'])
    
    # Collect results per model
    optuna_results = []
    
    # For each model, compute before/after tuning metrics and record time
    for model_name, model in MODELS.items():
        # Before tuning
        cvb = cross_validate(model, X_top, y_top, cv=cv,
                             scoring=SCORING, n_jobs=-1,
                             return_train_score=False)
        before = {f"{m}(Before)": cvb[f"test_{m}"].mean() for m in SCORING}
        
        # Optuna tuning
        study = optuna.create_study(direction='maximize')
        t0    = time.time()
        study.optimize(make_objective(X_top, y_top, model_name), n_trials=5)
        elapsed = time.time() - t0
        best_params = study.best_params
        
        # After tuning
        tuned = model.set_params(**best_params)
        cva   = cross_validate(tuned, X_top, y_top,
                               cv=cv, scoring=SCORING,
                               n_jobs=-1, return_train_score=False)
        after = {f"{m}(After)": cva[f"test_{m}"].mean() for m in SCORING}
        
        optuna_results.append({
            'Model'          : model_name,
            **before,
            **after,
            'Best Parameters': best_params,
            'Time (s)'       : elapsed
        })
    
    # Persist per-dataset Optuna results
    df_tune = pd.DataFrame(optuna_results).set_index('Model')
    outputs[ds_key]['optuna_results'] = df_tune

# Save Optuna tuning results to Excel
with pd.ExcelWriter("df_optuna.xlsx", engine="openpyxl") as writer:
    for i, (name, _) in enumerate(datasets.items(), start=1):
        sheet = f"{i}_{name}_{DATASET}"[:31]
        outputs[f"dataset_{i}"]['optuna_results'].to_excel(
            writer, sheet_name=sheet, index=True
        )

In [ ]:
# Hyperparameter optimization: Grid Search


# CV configuration
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Parameter grids per model
PARAM_GRIDS = {
    'KNN':  {'n_neighbors': [3, 5, 7],            'weights': ['uniform', 'distance']},
    'LR':   {'C': [0.01, 0.1, 1, 10]},
    'LDA':  {'solver': ['svd']},
    'NB':   {'var_smoothing': [1e-12, 1e-10, 1e-8]},
    'SVM':  {'C': [0.1, 1, 10],                    'gamma': ['scale', 'auto']},
    'RF':   {'n_estimators': [100, 200],           'max_depth': [None, 5, 10]},
    'GB':   {'n_estimators': [100, 200],           'learning_rate': [0.01, 0.1], 'max_depth': [3, 5]},
    'XGB':  {'n_estimators': [100, 200],           'learning_rate': [0.01, 0.1], 'max_depth': [3, 5]},
    'LGBM': {'n_estimators': [100, 200],           'num_leaves': [31, 64], 'learning_rate': [0.01, 0.1]},
    'MLP':  {'hidden_layer_sizes': [(50,), (100,), (50, 50)],
             'alpha': [1e-4, 1e-3, 1e-2],
             'learning_rate_init': [1e-3, 1e-2, 1e-1]}
}

# Grid Search per dataset
for i, (name, _) in enumerate(datasets.items(), start=1):
    ds_key = f"dataset_{i}"
    
    # Use the same data view chosen previously
    df_used = {
        'original': outputs[ds_key]['original'],
        'pca'     : outputs[ds_key]['pca_scores'],
        'fa'      : outputs[ds_key]['fa_scores']
    }[DATASET]
    design_df = outputs[ds_key]['design']
    best_idx  = outputs[ds_key]['best_iteration'].at[0, 'Original Index']
    
    # Extract features from the best iteration
    feat_cols = [c for c in design_df.columns if c != 'Iteration']
    selected  = [c for c in feat_cols if design_df.at[best_idx, c] == 1]
    
    X_top = df_used.drop(columns='y')[selected]
    y_top = LabelEncoder().fit_transform(df_used['y'])
    
    grid_results = []
    
    # Loop over models
    for model_name, model in MODELS.items():
        # Before tuning (baseline CV metrics)
        cvb = cross_validate(
            model, X_top, y_top,
            cv=cv, scoring=SCORING,
            n_jobs=-1, return_train_score=False
        )
        before = {f"{m}(Before)": cvb[f"test_{m}"].mean() for m in SCORING}
        
        # Grid Search (optimize weighted F1)
        grid = GridSearchCV(
            model,
            PARAM_GRIDS[model_name],
            scoring='f1_weighted',
            cv=cv,
            n_jobs=-1,
            refit=True
        )
        t0 = time.time()
        grid.fit(X_top, y_top)
        elapsed = time.time() - t0
        best_params = grid.best_params_
        
        # After tuning (CV with best estimator)
        tuned = grid.best_estimator_
        cva   = cross_validate(
            tuned, X_top, y_top,
            cv=cv, scoring=SCORING,
            n_jobs=-1, return_train_score=False
        )
        after = {f"{m}(After)": cva[f"test_{m}"].mean() for m in SCORING}
        
        # Compile row
        row = {
            'Model'          : model_name,
            **before,
            **after,
            'Best Parameters': best_params,
            'Time (s)'       : elapsed
        }
        grid_results.append(row)
    
    # Build DataFrame and store in outputs
    df_gs = pd.DataFrame(grid_results).set_index('Model')
    outputs[ds_key]['grid_results'] = df_gs

# Persist Grid Search results to Excel
with pd.ExcelWriter("df_grid_search.xlsx", engine="openpyxl") as writer:
    for i, (name, _) in enumerate(datasets.items(), start=1):
        sheet = f"{i}_{name}_{DATASET}"[:31]
        outputs[f"dataset_{i}"]['grid_results'].to_excel(writer, sheet_name=sheet)

In [15]:
# Statistical tests

# Configuration
PATHS = {
    "fa_results":  "fa_df_ml_results.xlsx",
    "pca_results": "pca_df_ml_results.xlsx",
    "orig_results":"original_df_ml_results.xlsx",
    "fa_times":    "fa_df_ml_times.xlsx",
    "pca_times":   "pca_df_ml_times.xlsx",
    "orig_times":  "original_df_ml_times.xlsx",
}
PREPROS     = ["FA", "PCA", "Original"]
RESULT_KEYS = {"FA": "fa_results", "PCA": "pca_results", "Original": "orig_results"}
TIME_KEYS   = {"FA": "fa_times",   "PCA": "pca_times",   "Original": "orig_times"}
METRICS     = ["A", "P", "R", "F1"]
PREF_SUFFIX = {
    "fa_results":  ["fa"],
    "pca_results": ["pca"],
    "orig_results":["orig", "original", "org"],
    "fa_times":    ["fa"],
    "pca_times":   ["pca"],
    "orig_times":  ["orig", "original", "org"],
}

# Bootstrap setup
N_BOOT   = 2000
BOOT_SEED = 2025
ALPHA     = 0.05

# Worksheet helpers
def sheet_prefix(name: str) -> str:
    return name.rsplit("_", 1)[0].strip() if "_" in name else name.strip()

def sheet_suffix(name: str) -> str:
    return name.rsplit("_", 1)[1].strip().lower() if "_" in name else ""

def choose_sheet_for_base(cands: list[str], prefer_suffixes: list[str] | None = None) -> str | None:
    if not cands:
        return None
    if prefer_suffixes:
        for pref in prefer_suffixes:
            for c in cands:
                if sheet_suffix(c) == pref.lower():
                    return c
    return cands[0]

def read_sheet_by_base(book_path: Path, base: str, prefer_suffix: list[str] | None) -> pd.DataFrame:
    xls = pd.ExcelFile(book_path)
    cands = [s for s in xls.sheet_names if sheet_prefix(s) == base]
    if not cands:
        raise ValueError(f"Sheet with prefix '{base}' not found in '{book_path.name}'. Sheets: {xls.sheet_names}")
    chosen = choose_sheet_for_base(cands, prefer_suffix)
    return pd.read_excel(book_path, sheet_name=chosen)

# Effect Sizes and Utilities
def epsilon_squared_kw(H: float, k: int, N: int) -> float:
    if N <= k:
        return np.nan
    return float(np.clip((H - k + 1) / (N - k), 0.0, 1.0))

def cliffs_delta(x: np.ndarray, y: np.ndarray) -> float:
    x = np.asarray(x); y = np.asarray(y)
    x = x[~np.isnan(x)]; y = y[~np.isnan(y)]
    if x.size == 0 or y.size == 0:
        return np.nan
    greater = 0; less = 0
    for xi in x:
        greater += np.sum(xi > y)
        less    += np.sum(xi < y)
    nxy = x.size * y.size
    return (greater - less) / nxy if nxy > 0 else np.nan

def rank_biserial_from_mw(u_stat: float, n1: int, n2: int) -> float:
    if n1 <= 0 or n2 <= 0:
        return np.nan
    return 1.0 - 2.0 * (u_stat / (n1 * n2))

def holm_correction(pvals: list[float]) -> list[float]:
    m = len(pvals)
    order = np.argsort(pvals)
    adj = np.empty(m, dtype=float)
    prev = 0.0
    for i, idx in enumerate(order):
        adj_p = (m - i) * pvals[idx]
        adj[idx] = max(adj_p, prev)
        prev = adj[idx]
    return np.minimum(adj, 1.0).tolist()

# Diagnostics (normality/homoscedasticity)
def shapiro_safe(x: np.ndarray) -> float:
    """Return Shapiro–Wilk p-value, or NaN if out of supported range (n<3 or n>5000)."""
    x = np.asarray(x); x = x[~np.isnan(x)]
    n = x.size
    if n < 3 or n > 5000:
        return np.nan
    try:
        return float(shapiro(x).pvalue)
    except Exception:
        return np.nan

def levene_safe(groups: list[np.ndarray], center: str = "median") -> float:
    """Return Levene p-value (robust) for 2+ groups; NaN if any empty."""
    cleaned = [np.asarray(g)[~np.isnan(g)] for g in groups]
    if any(len(g) == 0 for g in cleaned) or len(cleaned) < 2:
        return np.nan
    try:
        return float(levene(*cleaned, center=center).pvalue)
    except Exception:
        return np.nan

# Bootstrap CIs
rng = np.random.default_rng(BOOT_SEED)

def bootstrap_ci_diff_median(x: np.ndarray, y: np.ndarray, n_boot: int = N_BOOT, alpha: float = ALPHA) -> tuple[float,float,float]:
    """Return (median_diff, ci_low, ci_high)."""
    x = np.asarray(x)[~np.isnan(x)]
    y = np.asarray(y)[~np.isnan(y)]
    if x.size == 0 or y.size == 0:
        return (np.nan, np.nan, np.nan)
    diff = float(np.median(x) - np.median(y))
    bx = rng.choice(x, size=(n_boot, x.size), replace=True)
    by = rng.choice(y, size=(n_boot, y.size), replace=True)
    diffs = np.median(bx, axis=1) - np.median(by, axis=1)
    lo, hi = np.percentile(diffs, [100*alpha/2, 100*(1-alpha/2)])
    return (diff, float(lo), float(hi))

def bootstrap_ci_cliffs_delta(x: np.ndarray, y: np.ndarray, n_boot: int = N_BOOT, alpha: float = ALPHA) -> tuple[float,float,float]:
    """Return (delta, ci_low, ci_high)."""
    x = np.asarray(x)[~np.isnan(x)]
    y = np.asarray(y)[~np.isnan(y)]
    if x.size == 0 or y.size == 0:
        return (np.nan, np.nan, np.nan)
    base = cliffs_delta(x, y)
    bx_idx = rng.integers(0, len(x), size=(n_boot, len(x)))
    by_idx = rng.integers(0, len(y), size=(n_boot, len(y)))
    deltas = []
    for i in range(n_boot):
        deltas.append(cliffs_delta(x[bx_idx[i]], y[by_idx[i]]))
    lo, hi = np.percentile(deltas, [100*alpha/2, 100*(1-alpha/2)])
    return (float(base), float(lo), float(hi))

# Parsing and Aggregation
def parse_algos_from_results_columns(columns: list[str]) -> dict[str, list[str]]:
    pat = re.compile(r"^(?P<algo>.+)\((?P<m>A|P|R|F1)\)$", flags=re.I)
    temp = {}
    for col in columns:
        m = pat.match(str(col).strip())
        if not m:
            continue
        algo = m.group("algo").strip()
        met  = m.group("m").upper()
        temp.setdefault(algo, {})[met] = col
    algo2cols = {}
    for algo, d in temp.items():
        if all(m in d for m in METRICS):
            algo2cols[algo] = [d[m] for m in METRICS]
    return algo2cols

def average_metrics_per_algo(df: pd.DataFrame, algo2cols: dict[str, list[str]]) -> pd.DataFrame:
    out = {}
    for algo, cols in algo2cols.items():
        out[algo] = df[cols].astype(float).mean(axis=1, skipna=True)
    return pd.DataFrame(out)

def extract_times_wide(df: pd.DataFrame) -> pd.DataFrame:
    time_cols = [c for c in df.columns if str(c).endswith("_time")]
    if not time_cols:
        time_cols = [c for c in df.columns if "time" in str(c).lower() or "tempo" in str(c).lower()]
    rename = {c: re.sub(r"_time$", "", str(c)) for c in time_cols}
    return df[time_cols].astype(float).rename(columns=rename)

# Tests (KW, MW, and Diagnostics)
def kw_and_diagnostics(groups: dict[str, np.ndarray]) -> dict:
    """Kruskal–Wallis + epsilon² + diagnostics (per-group Shapiro; Levene)."""
    labels = list(groups.keys())
    data = [np.asarray(groups[l], dtype=float) for l in labels]
    data = [d[~np.isnan(d)] for d in data]
    sizes = [len(d) for d in data]
    N = sum(sizes); k = len(labels)

    res = {"KW_H": np.nan, "KW_p": np.nan, "KW_k": k, "KW_N": N, "KW_epsilon_sq": np.nan}
    if any(s == 0 for s in sizes) or k < 2:
        # Still compute diagnostics
        for lab, arr in zip(labels, data):
            res[f"SW_p_{lab}"] = shapiro_safe(arr)
        res["Levene_p"] = levene_safe(data, center="median")
        return res

    H, p_kw = kruskal(*data, nan_policy="omit")
    eps2 = epsilon_squared_kw(H, k, N)
    res.update({"KW_H": H, "KW_p": p_kw, "KW_epsilon_sq": eps2})

    # Diagnostics
    for lab, arr in zip(labels, data):
        res[f"SW_p_{lab}"] = shapiro_safe(arr)
    res["Levene_p"] = levene_safe(data, center="median")
    return res

def mw_posthoc_with_boot(groups: dict[str, np.ndarray]) -> pd.DataFrame:
    """Mann–Whitney U + effect sizes + CIs (median diff & Cliff's delta) + Holm."""
    labels = list(groups.keys())
    data = {l: np.asarray(groups[l], dtype=float)[~np.isnan(groups[l])] for l in labels}

    pairs = list(itertools.combinations(labels, 2))
    rows = []
    for a, b in pairs:
        x, y = data[a], data[b]
        if len(x) == 0 or len(y) == 0:
            rows.append({
                "pair": f"{a} vs {b}", "U": np.nan, "p": np.nan, "p_adj_holm": np.nan,
                "cliffs_delta": np.nan, "cliffs_delta_ci_low": np.nan, "cliffs_delta_ci_high": np.nan,
                "rank_biserial": np.nan,
                "median_diff": np.nan, "median_diff_ci_low": np.nan, "median_diff_ci_high": np.nan,
                "direction": ""
            })
            continue

        mw = mannwhitneyu(x, y, alternative="two-sided")
        u_stat, pval = mw.statistic, mw.pvalue
        delta, d_lo, d_hi = bootstrap_ci_cliffs_delta(x, y, n_boot=N_BOOT, alpha=ALPHA)
        rb = rank_biserial_from_mw(u_stat, len(x), len(y))
        mdiff, m_lo, m_hi = bootstrap_ci_diff_median(x, y, n_boot=N_BOOT, alpha=ALPHA)
        direction = f"{a}>{b}" if np.nanmedian(x) > np.nanmedian(y) else f"{b}>{a}"

        rows.append({
            "pair": f"{a} vs {b}",
            "U": u_stat, "p": pval,
            "cliffs_delta": delta, "cliffs_delta_ci_low": d_lo, "cliffs_delta_ci_high": d_hi,
            "rank_biserial": rb,
            "median_diff": mdiff, "median_diff_ci_low": m_lo, "median_diff_ci_high": m_hi,
            "direction": direction
        })

    df = pd.DataFrame(rows)
    if not df.empty:
        df["p_adj_holm"] = holm_correction(df["p"].fillna(1.0).tolist())
    return df

# Analyses: Performance and Time
def analyze_results_for_base(book_paths: dict[str, Path], base: str) -> tuple[pd.DataFrame, pd.DataFrame]:
    dfs = {}
    algo_sets = []
    for label, key in [("FA", "fa_results"), ("PCA", "pca_results"), ("Original", "orig_results")]:
        df = read_sheet_by_base(book_paths[key], base, PREF_SUFFIX.get(key))
        algo2cols = parse_algos_from_results_columns(list(df.columns))
        algo_sets.append(set(algo2cols.keys()))
        dfs[label] = average_metrics_per_algo(df, algo2cols)

    common_algos = set.intersection(*algo_sets) if algo_sets else set()
    stats_rows = []
    pairs_all = []

    for algo in sorted(common_algos):
        groups = {pre: dfs[pre][algo].values for pre in PREPROS}
        # KW + diagnostics
        kw_diag = kw_and_diagnostics(groups)
        stats_rows.append({
            "dataset_base": base, "section": "results", "level": "per-algorithm", "target": algo, **kw_diag
        })
        # Post-hoc + bootstrap CIs
        df_pairs = mw_posthoc_with_boot(groups)
        if not df_pairs.empty:
            df_pairs.insert(0, "dataset_base", base)
            df_pairs.insert(1, "section", "results")
            df_pairs.insert(2, "level", "per-algorithm")
            df_pairs.insert(3, "target", algo)
            pairs_all.append(df_pairs)

    # Overview (mean across common algorithms)
    if common_algos:
        mean_by_pre = {}
        for pre in PREPROS:
            mean_by_pre[pre] = dfs[pre][sorted(common_algos)].mean(axis=1, skipna=True).values
        kw_diag = kw_and_diagnostics(mean_by_pre)
        stats_rows.append({
            "dataset_base": base, "section": "results", "level": "overview", "target": "MEAN_OVER_ALGOS", **kw_diag
        })
        df_pairs = mw_posthoc_with_boot(mean_by_pre)
        if not df_pairs.empty:
            df_pairs.insert(0, "dataset_base", base)
            df_pairs.insert(1, "section", "results")
            df_pairs.insert(2, "level", "overview")
            df_pairs.insert(3, "target", "MEAN_OVER_ALGOS")
            pairs_all.append(df_pairs)

    df_stats = pd.DataFrame(stats_rows)
    df_pairs = pd.concat(pairs_all, ignore_index=True, sort=False) if pairs_all else pd.DataFrame()
    return df_stats, df_pairs

def analyze_times_for_base(book_paths: dict[str, Path], base: str) -> tuple[pd.DataFrame, pd.DataFrame]:
    dfs = {}
    algo_sets = []
    for label, key in [("FA", "fa_times"), ("PCA", "pca_times"), ("Original", "orig_times")]:
        df = read_sheet_by_base(book_paths[key], base, PREF_SUFFIX.get(key))
        tdf = extract_times_wide(df)
        algo_sets.append(set(tdf.columns))
        dfs[label] = tdf

    common_algos = set.intersection(*algo_sets) if algo_sets else set()
    stats_rows = []
    pairs_all = []

    for algo in sorted(common_algos):
        groups = {pre: dfs[pre][algo].values for pre in PREPROS}
        kw_diag = kw_and_diagnostics(groups)
        stats_rows.append({
            "dataset_base": base, "section": "times", "level": "per-algorithm_time", "target": algo, **kw_diag
        })
        df_pairs = mw_posthoc_with_boot(groups)
        if not df_pairs.empty:
            df_pairs.insert(0, "dataset_base", base)
            df_pairs.insert(1, "section", "times")
            df_pairs.insert(2, "level", "per-algorithm_time")
            df_pairs.insert(3, "target", algo)
            pairs_all.append(df_pairs)

    if common_algos:
        mean_by_pre = {}
        for pre in PREPROS:
            mean_by_pre[pre] = dfs[pre][sorted(common_algos)].mean(axis=1, skipna=True).values
        kw_diag = kw_and_diagnostics(mean_by_pre)
        stats_rows.append({
            "dataset_base": base, "section": "times", "level": "overview_time", "target": "MEAN_OVER_ALGOS_TIME", **kw_diag
        })
        df_pairs = mw_posthoc_with_boot(mean_by_pre)
        if not df_pairs.empty:
            df_pairs.insert(0, "dataset_base", base)
            df_pairs.insert(1, "section", "times")
            df_pairs.insert(2, "level", "overview_time")
            df_pairs.insert(3, "target", "MEAN_OVER_ALGOS_TIME")
            pairs_all.append(df_pairs)

    df_stats = pd.DataFrame(stats_rows)
    df_pairs = pd.concat(pairs_all, ignore_index=True, sort=False) if pairs_all else pd.DataFrame()
    return df_stats, df_pairs

# Save to Excel (One sheet per dataset)
def safe_sheet_name(name: str, fallback: str = "sheet") -> str:
    bad = r'[:\\/?*\[\]]'
    name = re.sub(bad, "_", str(name)).strip().rstrip(".")
    return (name or fallback)[:31]

def run_all_to_excel():
    base_dir = Path(".")
    book_paths = {k: base_dir / v for k, v in PATHS.items()}
    for k, p in book_paths.items():
        if not p.exists():
            print(f"[WARNING] File not found: {p.resolve()} (key: {k})")

    def collect_bases(keys):
        bases_sets = []
        for key in keys:
            xls = pd.ExcelFile(book_paths[key])
            bases_sets.append({sheet_prefix(s) for s in xls.sheet_names})
        return sorted(list(set.intersection(*bases_sets))) if bases_sets else []

    bases_results = collect_bases(["fa_results","pca_results","orig_results"])
    bases_times   = collect_bases(["fa_times","pca_times","orig_times"])

    print(f"Datasets (RESULTS): {len(bases_results)} | Datasets (TIMES): {len(bases_times)}")

    res_stats_by_base, res_pairs_by_base = {}, {}
    tim_stats_by_base, tim_pairs_by_base = {}, {}

    for base in bases_results:
        print(f"[results] {base}")
        res_stats, res_pairs = analyze_results_for_base(book_paths, base)
        res_stats_by_base[base] = res_stats
        res_pairs_by_base[base] = res_pairs

    for base in bases_times:
        print(f"[times] {base}")
        t_stats, t_pairs = analyze_times_for_base(book_paths, base)
        tim_stats_by_base[base] = t_stats
        tim_pairs_by_base[base] = t_pairs

    # results_stats_summary.xlsx (Includes Shapiro and Levene)
    with pd.ExcelWriter("results_stats_summary.xlsx", engine="openpyxl") as writer:
        for base, df in res_stats_by_base.items():
            diag_cols = [c for c in df.columns if c.startswith("SW_p_")] + (["Levene_p"] if "Levene_p" in df.columns else [])
            core = ["dataset_base","section","level","target","KW_H","KW_p","KW_k","KW_N","KW_epsilon_sq"]
            cols_order = [c for c in core if c in df.columns] + diag_cols + [c for c in df.columns if c not in core + diag_cols]
            df[cols_order].to_excel(writer, sheet_name=safe_sheet_name(base), index=False)

    # results_posthoc_pairs.xlsx (With CIs)
    with pd.ExcelWriter("results_posthoc_pairs.xlsx", engine="openpyxl") as writer:
        for base, df in res_pairs_by_base.items():
            core = ["dataset_base","section","level","target","pair","U","p","p_adj_holm",
                    "cliffs_delta","cliffs_delta_ci_low","cliffs_delta_ci_high",
                    "rank_biserial",
                    "median_diff","median_diff_ci_low","median_diff_ci_high",
                    "direction"]
            cols_order = [c for c in core if c in df.columns] + [c for c in df.columns if c not in core]
            df[cols_order].to_excel(writer, sheet_name=safe_sheet_name(base), index=False)

    # times_stats_summary.xlsx (Includes Shapiro and Levene)
    with pd.ExcelWriter("times_stats_summary.xlsx", engine="openpyxl") as writer:
        for base, df in tim_stats_by_base.items():
            diag_cols = [c for c in df.columns if c.startswith("SW_p_")] + (["Levene_p"] if "Levene_p" in df.columns else [])
            core = ["dataset_base","section","level","target","KW_H","KW_p","KW_k","KW_N","KW_epsilon_sq"]
            cols_order = [c for c in core if c in df.columns] + diag_cols + [c for c in df.columns if c not in core + diag_cols]
            df[cols_order].to_excel(writer, sheet_name=safe_sheet_name(base), index=False)

    # times_posthoc_pairs.xlsx (With CIs)
    with pd.ExcelWriter("times_posthoc_pairs.xlsx", engine="openpyxl") as writer:
        for base, df in tim_pairs_by_base.items():
            core = ["dataset_base","section","level","target","pair","U","p","p_adj_holm",
                    "cliffs_delta","cliffs_delta_ci_low","cliffs_delta_ci_high",
                    "rank_biserial",
                    "median_diff","median_diff_ci_low","median_diff_ci_high",
                    "direction"]
            cols_order = [c for c in core if c in df.columns] + [c for c in df.columns if c not in core]
            df[cols_order].to_excel(writer, sheet_name=safe_sheet_name(base), index=False)

    print("\n=== Generated Excel files ===")
    print("- results_stats_summary.xlsx")
    print("- results_posthoc_pairs.xlsx")
    print("- times_stats_summary.xlsx")
    print("- times_posthoc_pairs.xlsx")
    print("\nNew columns: SW_p_* (Shapiro per group), Levene_p, median_diff ± CI, cliffs_delta ± CI.")

# Run
run_all_to_excel()

Datasets (RESULTS): 10 | Datasets (TIMES): 10
[results] 10_waveform_database_generator
[results] 1_statlog_australian_credit
[results] 2_wine_quality_red
[results] 3_breast_cancer_wisconsin
[results] 4_burst_header_packet
[results] 5_seismic_bumps
[results] 6_iranian_churn
[results] 7_qsar_biodegradation
[results] 8_image_segmentation
[results] 9_steel_plates_faults
[times] 10_waveform_database_generator
[times] 1_statlog_australian_credit
[times] 2_wine_quality_red
[times] 3_breast_cancer_wisconsin
[times] 4_burst_header_packet
[times] 5_seismic_bumps
[times] 6_iranian_churn
[times] 7_qsar_biodegradation
[times] 8_image_segmentation
[times] 9_steel_plates_faults

=== Excel gerados ===
- results_stats_summary.xlsx
- results_posthoc_pairs.xlsx
- times_stats_summary.xlsx
- times_posthoc_pairs.xlsx

Colunas novas: SW_p_* (Shapiro por grupo), Levene_p, median_diff ± CI, cliffs_delta ± CI.
